In [2]:
%%time
import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role
role=get_execution_role()

CPU times: user 150 ms, sys: 6.7 ms, total: 157 ms
Wall time: 234 ms


In [3]:
%%time
tfmodel=TensorFlow(entry_point='cats_n_dogs_script.py',
                             role=role,
                             instance_count=1,
                             instance_type='ml.m5.large',
                             framework_version='2.4.1',
                             py_version='py37',
                   script_mode=True,
                  hyperparameters={'epochs': 1})
#                              distribution={'parameter_server': {'enabled': True}})

CPU times: user 55.5 ms, sys: 7.71 ms, total: 63.2 ms
Wall time: 68.3 ms


In [4]:
%%time
tfmodel.fit(
    {'training':'s3://sagemaker-us-east-1-032934527328/cats_dogs_prediction/',
     'validation':'s3://sagemaker-us-east-1-032934527328/cats_dogs_prediction/'
    })

2021-04-19 21:52:14 Starting - Starting the training job...
2021-04-19 21:52:18 Starting - Launching requested ML instancesProfilerReport-1618869134: InProgress
......
2021-04-19 21:53:43 Starting - Preparing the instances for training......
2021-04-19 21:54:44 Downloading - Downloading input data.........
2021-04-19 21:56:07 Training - Training image download completed. Training in progress.2021-04-19 21:56:08.749247: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-04-19 21:56:08.766636: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-04-19 21:56:09.077294: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-04-19 21:56:18,617 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-04-19 21


2021-04-19 21:57:15 Uploading - Uploading generated training model
2021-04-19 21:57:15 Completed - Training job completed
Training seconds: 151
Billable seconds: 151
CPU times: user 719 ms, sys: 41.3 ms, total: 761 ms
Wall time: 5min 13s


In [5]:
tfmodel.model_dir

's3://sagemaker-us-east-1-032934527328/tensorflow-training-2021-04-19-21-52-14-081/model'

In [ ]:
# 2 ways to deploy model 
# 1. directly from model.tar.gz - see next two cells
# 2. call deploy or transform on the estimator object( IMPORTANT !! model.save(args.model_dir +'/1') otherwise the deploy method doesnt directl work, you can use 1 bove)

In [8]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(model_data='s3://sagemaker-us-east-1-032934527328/tensorflow-training-2021-04-19-21-52-14-081/output/model.tar.gz',
                        role=role,framework_version='2.4.1'
                           )

predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!

In [ ]:
from sagemaker.tensorflow import TensorFlowModel

anothermodel = TensorFlowModel(model_data='s3://sagemaker-us-east-1-032934527328/tensorflow-training-2021-04-18-12-47-27-360/model.tar.gz',
                               role=role,framework_version='2.4.1'
                           )

predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----

In [18]:
# Batch transform
tfmodel_predictor=tfmodel.deploy(initial_instance_count=1,instance_type='ml.m5.large')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!

In [26]:
import numpy as np
y=np.random.randint(0, high=255, size=(1,150,150,3), dtype=int)
print(y.shape)

(1, 150, 150, 3)


In [27]:
tfmodel_predictor.predict(y)

{'predictions': [[0.198511407, 0.801488578]]}

In [28]:
tfmodel_predictor.delete_endpoint()